#  Lab เสริม - เครือข่ายประสาทเทียมแบบง่าย

ใน Lab นี้ เราจะสร้างเครือข่ายประสาทเทียมขนาดเล็กโดยใช้ Numpy มันจะเป็นเครือข่าย "การคั่วกาแฟ" เดียวกันกับที่คุณได้พัฒนาใน Tensorflow
   <center> <img  src="./images/C2_W1_CoffeeRoasting.png" width="400" />   <center/>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from lab_utils_common import dlc, sigmoid
from lab_coffee_utils import load_coffee_data, plt_roast, plt_prob, plt_layer, plt_network, plt_output_unit
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

## ชุดข้อมูล
นี่เป็นชุดข้อมูลเดียวกับในห้องปฏิบัติการก่อนหน้านี้

In [ ]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

มาพล็อตข้อมูลการคั่วกาแฟด้านล่างกันครับ คุณสมบัติทั้งสองคือ อุณหภูมิเป็นองศาเซลเซียสและระยะเวลาเป็นนาที. [Coffee Roasting at Home](https://www.merchantsofgreencoffee.com/how-to-roast-green-coffee-in-your-oven/) แนะนำว่าระยะเวลาควรอยู่ระหว่าง 12 ถึง 15 นาที ในขณะที่อุณหภูมิควรอยู่ระหว่าง 175 ถึง 260 องศาเซลเซียส แน่นอนว่าเมื่ออุณหภูมิสูงขึ้น ระยะเวลาก็ควรลดลง

In [ ]:
plt_roast(X,Y)

### Normalize Data: "ปรับมาตรฐานข้อมูล" หรือ "มาตรฐานข้อมูล"
เพื่อให้สอดคล้องกับการทดลองก่อนหน้า เราจะทำการปรับมาตรฐานข้อมูล กรุณาอ้างอิงการทดลองก่อนหน้าสำหรับรายละเอียดเพิ่มเติม

In [ ]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

## Numpy Model (Forward Prop in NumPy)
<center> <img  src="./images/C2_W1_RoastingNetwork.PNG" width="200" />   <center/>  
มาสร้าง "เครือข่ายการคั่วกาแฟ" ตามที่ได้กล่าวถึงในบรรยายกันเถอะ เครือข่ายนี้จะมีสองชั้น โดยใช้ฟังก์ชันการกระตุ้นซิกมอยด์ (sigmoid activations)

ตามที่อธิบายไว้ในบรรยาย คุณสามารถสร้างเลเยอร์หนาแน่นของคุณเองโดยใช้ NumPy จากนั้นสามารถนำมาใช้สร้างเครือข่ายประสาทเทียมหลายชั้นได้

<img src="images/C2_W1_dense3.png" width="600" height="450">

ในห้องปฏิบัติการเสริมครั้งแรก คุณได้สร้างเซลล์ประสาทเทียม (neuron) ใน NumPy และ TensorFlow และสังเกตเห็นความคล้ายคลึงกัน ชั้นหนึ่งประกอบด้วยเซลล์ประสาทเทียม/ยูนิตหลายตัว ตามที่อธิบายไว้ในบรรยาย สามารถใช้ลูป for เพื่อเยี่ยมชมแต่ละยูนิต  (`j`) ในชั้นและทำผลคูณดอทของน้ำหนักสำหรับยูนิตนั้น(`W[:,j]`) และบวกไบแอสสำหรับยูนิต (`b[j]`) เพื่อสร้าง `z`. จากนั้นสามารถนำฟังก์ชันการกระตุ้น `g(z)` ไปใช้กับผลลัพธ์นั้นได้ มาลองสร้างซับรูทีน "ชั้นหนาแน่น" ด้านล่างกัน"


ขั้นแรก คุณจะต้องนิยามฟังก์ชันการกระตุ้น (activation function) `g()`.  โดยคุณจะใช้ฟังก์ชัน  `sigmoid()` ซึ่งมีการเขียนไว้ให้คุณแล้วภายในไฟล์ `lab_utils_common.py` นอกโน้ตบุ๊คเล่ม์นี้

In [ ]:
# Define the activation function
g = sigmoid

ต่อไป คุณจะกำหนดฟังก์ชัน `my_dense()` ซึ่งคำนวณการกระตุ้นของเลเยอร์หนาแน่น (dense layer).

In [ ]:
def my_dense(a_in, W, b):
    """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example 
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units  
    Returns
      a_out (ndarray (j,))  : j units|
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):               
        w = W[:,j]                                    
        z = np.dot(w, a_in) + b[j]         
        a_out[j] = g(z)               
    return(a_out)

*"หมายเหตุ: คุณสามารถปรับใช้ฟังก์ชันข้างต้นเพื่อยอมรับ `g` เป็นพารามิเตอร์เพิ่มเติม (เช่น `my_dense(a_in, W, b, g)`ได้) อย่างไรก็ตาม ในโน้ตบุ๊กนี้ คุณจะใช้ฟังก์ชันการกระตุ้นชนิดเดียว (เช่น sigmoid) ดังนั้นจึงไม่จำเป็นต้องกำหนดเป็นค่าคงที่และกำหนดไว้ภายนอกฟังก์ชัน นั่นคือสิ่งที่คุณทำในโค้ดข้างต้น และทำให้การเรียกฟังก์ชันในเซลล์โค้ดถัดไปง่ายขึ้น เพียงแค่จำไว้ว่าการส่งผ่านเป็นพารามิเตอร์ก็เป็นการใช้งานที่ยอมรับได้เช่นกัน คุณจะเห็นในงานมอบหมายสัปดาห์นี้".*

"เซลล์ต่อไปนี้สร้างเครือข่ายประสาทเทียมสองชั้นโดยใช้ซับรูทีน `my_dense` ด้านบน

In [ ]:
def my_sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x,  W1, b1)
    a2 = my_dense(a1, W2, b2)
    return(a2)

เราสามารถคัดลอกน้ำหนักและความเอนเอียงที่ได้รับการฝึกอบรมจากห้องปฏิบัติการก่อนหน้านี้ใน TensorFlow

In [ ]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

### Predictions
<img align="left" src="./images/C2_W1_RoastingDecision.PNG"     style=" width:380px; padding: 10px 20px; " >

การใช้โมเดลที่ฝึกแล้วเพื่อทำนายผล
เมื่อคุณมีโมเดลที่ได้รับการฝึกแล้ว คุณสามารถใช้โมเดลนั้นเพื่อทำนายผลได้

โปรดจำไว้ว่าผลลัพธ์ของโมเดลของเราคือความน่าจะเป็น

ในกรณีนี้ ความน่าจะเป็นของการคั่วกาแฟที่ดี

เพื่อตัดสินใจ คุณต้องนำความน่าจะเป็นไปใช้กับเกณฑ์ (threshold)

ในกรณีนี้ เราจะใช้ 0.5

เรามาเริ่มเขียนโปรแกรมแบบ Routine ที่คล้ายกับ `model.predict()`. ของ TensorFlow กันก่อน โปรแกรมนี้จะรับเมทริกซ์ $X$ ที่มีตัวอย่างทั้งหมด $m$ แถว และทำการทำนายโดยการรันโมเดล

In [ ]:
def my_predict(X, W1, b1, W2, b2):
    m = X.shape[0]
    p = np.zeros((m,1))
    for i in range(m):
        p[i,0] = my_sequential(X[i], W1, b1, W2, b2)
    return(p)

เราสามารถลองใช้กิจวัตรนี้กับตัวอย่างสองตัวอย่างได้ดังนี้:

In [ ]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

เพื่อแปลงความน่าจะเป็นให้เป็นการตัดสินใจ เราใช้เกณฑ์ (threshold):

In [ ]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

สามารถทำได้อย่างรวบรัดกว่านี้:


In [ ]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

## Network function

กราฟนี้แสดงการทำงานของเครือข่ายทั้งหมดและเหมือนกับผลลัพธ์ของ TensorFlow จากการทดลองก่อนหน้า

กราฟด้านซ้ายคือผลลัพธ์ดิบของเลเยอร์สุดท้ายที่แสดงโดยการแรเงาสีฟ้า ซึ่งทับซ้อนกับข้อมูลการฝึกที่แสดงโดย X และ O

กราฟด้านขวาคือผลลัพธ์ของเครือข่ายหลังจากผ่านเกณฑ์การตัดสินใจ X และ O ที่นี่สอดคล้องกับการตัดสินใจที่เครือข่ายทำ

In [ ]:
netf= lambda x : my_predict(norm_l(x),W1_tmp, b1_tmp, W2_tmp, b2_tmp)
plt_network(X,Y,netf)

## ขอแสดงความยินดี!

คุณได้สร้างเครือข่ายประสาทเทียมขนาดเล็กใน NumPy แล้ว

หวังว่าห้องปฏิบัติการนี้จะเผยให้เห็นฟังก์ชันที่ค่อนข้างง่ายและคุ้นเคยซึ่งประกอบเป็นชั้นในเครือข่ายประสาทเทียม